#  Zillow Project: Data Preparation
---

## MVP Objectives
- Remove/Impute missing values.
- Create functions to reproduce prepared dataset
- Move functions to a seperate file named `prepare.py`


In [1]:
# Import libraries to manipulate data structures and visualize numeric data.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in the zillow dataset and assign to a variable
df = pd.read_csv('zillow.csv')

In [3]:
# My first dataset with +2 million rows :)
df.shape

(2985217, 59)

In [4]:
# Initial look at our zillow dataset
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


Our dataset has a bunch of missing values! This is only the first 5 rows! Let's see how many values are missing in each column.

In [5]:
# Using `isnull()` and `notnull()` we can calculate the number of missing values and non-null values.
nulls = df.isnull().sum()
non_nulls = df.notnull().sum()

# Add missing values and non-null values together to get the total number values in each column.
total_values = nulls + non_nulls

# Create a variable to store the percentage of missing values in each column.
pct_missing = (nulls/total_values).sort_values(ascending=False)

# Perform formatting to clearly see the percentage of missing values in each column.
pct_missing_chart = pct_missing.apply("{0:.2%}".format)

# Display table to the user showing the percentage of missing values in each column.
print('Percentage of values missing per column')
print('-' * 39)
print(f"{pct_missing_chart}")

Percentage of values missing per column
---------------------------------------
storytypeid                     99.95%
basementsqft                    99.95%
yardbuildingsqft26              99.91%
fireplaceflag                   99.83%
architecturalstyletypeid        99.80%
typeconstructiontypeid          99.77%
finishedsquarefeet13            99.74%
buildingclasstypeid             99.57%
pooltypeid10                    99.43%
decktypeid                      99.42%
finishedsquarefeet6             99.28%
poolsizesum                     99.06%
pooltypeid2                     98.89%
hashottuborspa                  98.32%
taxdelinquencyflag              98.11%
taxdelinquencyyear              98.11%
yardbuildingsqft17              97.27%
finishedsquarefeet15            93.63%
finishedsquarefeet50            93.17%
finishedfloor1squarefeet        93.17%
fireplacecnt                    89.51%
threequarterbathnbr             89.40%
pooltypeid7                     83.05%
poolcnt                

Before we move ahead let's drop columns with more than __9.14%__ of values missing. Imputing values in columns with > 9.14% of values missing is a waste of time. We need to make our dataset _Robust™_.
- If we can impute values in columns: `lotsizesquarefeet`, `finishedsquarefeet12`, great. If not, we can drop them.

In [6]:
columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()

In [7]:
df_subset = df.drop(columns=columns_to_drop)

In [8]:
# My eyes are happy
df_subset.sample(10)

,id,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
2137613,2137613,11719960,2.0,2.0,2.0,1010.0,1010.0,6037.0,2.0,34023068.0,...,3101.0,95999.0,0.0,1913.0,82000.0,413000.0,2016.0,331000.0,5107.29,6.037223e+13
145942,145942,14186238,3.0,5.0,3.0,1737.0,1737.0,6059.0,3.0,33887191.0,...,1286.0,97078.0,6.0,1962.0,122412.0,220792.0,2016.0,98380.0,2748.82,6.059022e+13
2280626,2280626,14514609,0.0,0.0,NaN,NaN,NaN,6059.0,NaN,33746500.0,...,1286.0,97003.0,0.0,NaN,9066.0,9066.0,2016.0,NaN,22.60,6.059089e+13
2648145,2648145,12754348,1.0,2.0,1.0,752.0,752.0,6037.0,1.0,33895684.0,...,3101.0,96193.0,0.0,1949.0,62392.0,245831.0,2016.0,183439.0,3697.78,6.037553e+13
1351208,1351208,12780221,3.0,3.0,3.0,1708.0,1708.0,6037.0,3.0,33997884.0,...,3101.0,96169.0,0.0,1949.0,161735.0,348687.0,2016.0,186952.0,4493.21,6.037501e+13
814388,814388,12764428,1.0,2.0,1.0,1068.0,1068.0,6037.0,1.0,34055205.0,...,3101.0,96480.0,0.0,1942.0,141596.0,249110.0,2016.0,107514.0,3481.93,6.037434e+13
2575204,2575204,13841170,2.0,4.0,2.0,1539.0,1539.0,6059.0,2.0,33740116.0,...,1286.0,97003.0,7.0,1962.0,35478.0,55143.0,2016.0,19665.0,881.14,6.059075e+13
1901220,1901220,12321934,2.0,4.0,2.0,1229.0,1229.0,6037.0,2.0,33921767.0,...,3101.0,96125.0,0.0,1942.0,32413.0,64074.0,2016.0,31661.0,1111.78,6.037536e+13
1045735,1045735,10886085,1.0,4.0,1.0,1460.0,1460.0,6037.0,1.0,34182767.0,...,3101.0,96410.0,0.0,1939.0,211600.0,615000.0,2016.0,403400.0,7517.37,6.037124e+13
1919304,1919304,13063791,1.0,3.0,1.0,1004.0,1004.0,6037.0,1.0,34049210.0,...,3101.0,96506.0,0.0,1952.0,88272.0,189690.0,2016.0,101418.0,2425.01,6.037402e+13


Sweet, our data looks so much better. __We freed up $\approx$ .7+ GB of memory__! Let's press forward.

In [9]:
original_mem_usage = df.memory_usage(index=False).sum()/1_000_000_000
subset_mem_usage = df_subset.memory_usage(index=False).sum()/1_000_000_000

free_memory = original_mem_usage - subset_mem_usage
print(f"{free_memory:.2f}GB of free memory!")

0.79GB of free memory!


### Subset of Zillow data
- Dropped columns missing more than 9.14% of values

In [10]:
# Will return. Retrieving and reproducing data dictionary for these columns.
df_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 26 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   id                            int64  
 1   parcelid                      int64  
 2   bathroomcnt                   float64
 3   bedroomcnt                    float64
 4   calculatedbathnbr             float64
 5   calculatedfinishedsquarefeet  float64
 6   finishedsquarefeet12          float64
 7   fips                          float64
 8   fullbathcnt                   float64
 9   latitude                      float64
 10  longitude                     float64
 11  lotsizesquarefeet             float64
 12  propertycountylandusecode     object 
 13  propertylandusetypeid         float64
 14  rawcensustractandblock        float64
 15  regionidcity                  float64
 16  regionidcounty                float64
 17  regionidzip                   float64
 18  roomcnt               

In [11]:
# Create a function to clean our Zillow dataset
# Note: Create a wrangle function to acquire and prepare the dataset
def prepare_zillow(df):
    '''
    Signature: prepare_zillow(df) -> pandas.core.frame.DataFrame
    Docstring:
    Prepare the zillow dataset for data EDA

    Return DataFrame of zillow dataset

    Parameters
    ----------
    df : pandas.core.frame.DataFrame
    df is the Zillow dataset stored as `zillow.csv`

    Returns
    -------
    DataFrame of the zillow dataset


    Examples
    --------
    To copy the code hold ALT + SHIFT and drag your cursor from the first line to the last.

    >>> df = pd.read_csv('./data/raw/zillow.csv')
    >>>
    >>> nulls = df.isnull().sum()
    >>> non_nulls = df.notnull().sum()
    >>> total_values = nulls + non_nulls
    >>>
    >>> pct_missing = (nulls/total_values).sort_values(ascending=False)
    >>> pct_missing_chart = pct_missing.apply("{0:.2%}".format)
    >>>
    >>> print('Percentage of values missing per column')
    >>> print('-' * 39)
    >>> print(f"{pct_missing_chart}")
    '''
    
    # 1. Drop columns with missing values
    # Calculate the number of missing values and non-null values.
    nulls = df.isnull().sum()
    non_nulls = df.notnull().sum()

    # Get the total number values in each column.
    total_values = nulls + non_nulls

    # Create a variable to store the percentage of missing values in each column.
    # .3358 is a hard coded value from the original analysis.
    # To reproduce use the code in the docstring
    pct_missing = nulls/total_values
    columns_to_drop = pct_missing[pct_missing > .3358].index.to_list()
    
    # drop columns missing more than 33.58% of data.
    df = df.drop(columns=columns_to_drop)
    
    # drop duplicate columns and index column
    duplicate_columns_to_drop = ['calculatedbathnbr', 'finishedsquarefeet12', 'id']
    df = df.drop(columns=duplicate_columns_to_drop)
    
    df = df[df.propertylandusetypeid.isin([261, 262, 273, 279])]
    
    return df

In [12]:
test = prepare_zillow(df)

In [13]:
# Sweet, the function works!
test.sample(10)

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,...,regionidcounty,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock
510164,11215332,2.0,2.0,1362.0,6037.0,2.0,34566190.0,-118018468.0,7035.0,0101,...,3101.0,97330.0,0.0,1996.0,180000.0,225000.0,2016.0,45000.0,3496.21,6.037911e+13
1754632,12905055,1.0,3.0,924.0,6037.0,1.0,34108417.0,-117810420.0,5609.0,0100,...,3101.0,96513.0,0.0,1949.0,56807.0,243155.0,2016.0,186348.0,3177.21,6.037401e+13
2715382,10784341,2.0,4.0,1340.0,6037.0,2.0,34197039.0,-118544748.0,6112.0,0101,...,3101.0,96364.0,0.0,1950.0,35598.0,73885.0,2016.0,38287.0,1018.86,6.037133e+13
370670,14439827,4.0,3.0,2432.0,6059.0,4.0,33472548.0,-117639623.0,24150.0,122,...,1286.0,96983.0,7.0,1985.0,260320.0,435199.0,2016.0,174879.0,4353.30,6.059042e+13
2797288,12743478,1.0,3.0,1556.0,6037.0,1.0,33910460.0,-118083011.0,5426.0,0100,...,3101.0,96193.0,0.0,1950.0,32489.0,49445.0,2016.0,16956.0,947.36,6.037552e+13
1867933,11282561,2.0,3.0,1253.0,6037.0,2.0,34664296.0,-118118367.0,18299.0,0100,...,3101.0,97318.0,0.0,1959.0,52822.0,66019.0,2016.0,13197.0,1489.70,6.037901e+13
937039,14364768,3.0,3.0,2513.0,6059.0,3.0,33684543.0,-117630379.0,9900.0,122,...,1286.0,96989.0,0.0,1990.0,283286.0,439422.0,2016.0,156136.0,4164.30,6.059052e+13
2092486,11639133,5.0,5.0,4715.0,6037.0,5.0,34067522.0,-118411076.0,12277.0,0101,...,3101.0,96086.0,0.0,1938.0,541129.0,3787912.0,2016.0,3246783.0,44143.88,6.037701e+13
235838,14287422,1.5,2.0,1024.0,6059.0,1.0,33729200.0,-117816000.0,1151.0,122,...,1286.0,97016.0,5.0,1965.0,61541.0,183982.0,2016.0,122441.0,2428.78,6.059076e+13
1865604,17255301,2.5,4.0,2695.0,6111.0,2.0,34278017.0,-118754428.0,6968.0,1111,...,2061.0,97118.0,9.0,1968.0,221000.0,635000.0,2016.0,414000.0,7186.00,6.111008e+13


In [14]:
df = prepare_zillow(df)

In [15]:
df.shape

(2152864, 23)

In [16]:
df.fips.value_counts()

6037.0    1431812
6059.0     555077
6111.0     165975
Name: fips, dtype: int64

In [17]:
# Data dictionary template
numeric_columns = df.select_dtypes(exclude='O')

for column in numeric_columns.columns.to_list():
    print(column, numeric_columns[column].min(), numeric_columns[column].max())

parcelid 10711725 169601949
bathroomcnt 0.0 32.0
bedroomcnt 0.0 25.0
calculatedfinishedsquarefeet 1.0 952576.0
fips 6037.0 6111.0
fullbathcnt 1.0 32.0
latitude 33339912.0 34819650.0
longitude -119475780.0 -117554316.0
lotsizesquarefeet 100.0 371000512.0
propertylandusetypeid 261.0 279.0
rawcensustractandblock 60371011.101 61110091.003010996
regionidcity 3491.0 396556.0
regionidcounty 1286.0 3101.0
regionidzip 95982.0 399675.0
roomcnt 0.0 96.0
yearbuilt 1801.0 2016.0
structuretaxvaluedollarcnt 1.0 66404932.0
taxvaluedollarcnt 1.0 98428909.0
assessmentyear 2000.0 2016.0
landtaxvaluedollarcnt 1.0 88921951.0
taxamount 1.85 1337755.86
censustractandblock -1.0 483030105084015.0


square feet of home, number of bedrooms, and number of bathrooms.

In [18]:
def prepare_zillow_mvp(df):
    '''
    
    '''
    df = df[['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']]
    return df

In [19]:
df = pd.read_csv('zillow.csv')
df = prepare_zillow_mvp(df)

In [20]:
df.shape

(2985217, 4)

In [21]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
0,0.0,0.0,NaN,9.0
1,0.0,0.0,NaN,27516.0
2,0.0,0.0,73026.0,1434941.0
3,0.0,0.0,5068.0,1174475.0
4,0.0,0.0,1776.0,440101.0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 4 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bathroomcnt                   float64
 1   bedroomcnt                    float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
dtypes: float64(4)
memory usage: 91.1 MB


In [23]:
df.isna().sum()

bathroomcnt                      2957
bedroomcnt                       2945
calculatedfinishedsquarefeet    45097
taxvaluedollarcnt               34266
dtype: int64

In [24]:
df = df.dropna()

In [25]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
2,0.0,0.0,73026.0,1434941.0
3,0.0,0.0,5068.0,1174475.0
4,0.0,0.0,1776.0,440101.0
5,0.0,0.0,2400.0,287634.0
7,0.0,0.0,3611.0,698984.0


In [26]:
df = df[(df['bathroomcnt'] > 0) & (df['bathroomcnt'] > 0)]

In [29]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt
10,4.0,4.0,3095.0,192544.0
20,2.0,4.0,3633.0,296425.0
28,4.0,5.0,2865.0,831224.0
30,2.0,2.0,1090.0,357568.0
31,4.0,3.0,1620.0,847770.0


In [30]:
df['more_than_two_bath'] = (df.bathroomcnt > 2).astype('int')


In [31]:
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,more_than_two_bath
10,4.0,4.0,3095.0,192544.0,1
20,2.0,4.0,3633.0,296425.0,0
28,4.0,5.0,2865.0,831224.0,1
30,2.0,2.0,1090.0,357568.0,0
31,4.0,3.0,1620.0,847770.0,1
